In [1]:
import pandas as pd
from google.cloud import bigquery
import os
data = pd.read_csv(r'C:\Users\apoll\Downloads\weather_data_5_boroughs_daily (2).csv')

In [2]:
data.head()

,Station,latitude,longitude,Borough,City,State,ZipCode,WDate,Temperature_Max,Temperature_Avg,...,Humidity_M,Wdspeed_Max,Wdspeed_Avg,Wdspeed_M,Pressure_Max,Pressure_M,Precipitation_Total,Unnamed: 23,Unnamed: 24,Unnamed: 25
0,KNYBRONX14,40.8616,-73.8809,Bronx,Botanical Garden,NY,10458,1/1/2016,41.2,38.1,...,45,28.4,11.3,1.3,30.10,29.96,0.0,NaN,NaN,NaN
1,KNYBRONX14,40.8616,-73.8809,Bronx,Botanical Garden,NY,10458,1/2/2016,39.4,35.2,...,42,25.3,10.1,0.0,30.11,29.95,0.0,NaN,NaN,NaN
2,KNYBRONX14,40.8616,-73.8809,Bronx,Botanical Garden,NY,10458,1/3/2016,44.7,38.6,...,36,28.4,10.8,1.6,29.97,29.78,0.0,NaN,NaN,NaN
3,KNYBRONX14,40.8616,-73.8809,Bronx,Botanical Garden,NY,10458,1/4/2016,35.6,25.9,...,31,36.5,10.2,1.1,30.39,29.86,0.0,NaN,NaN,NaN
4,KNYBRONX14,40.8616,-73.8809,Bronx,Botanical Garden,NY,10458,1/5/2016,28.8,18.5,...,21,27.5,6.2,0.0,30.64,30.37,0.0,NaN,NaN,NaN


In [3]:
df = data.iloc[: , :-3]
df.drop(columns=['latitude','longitude'])

,Station,Borough,City,State,ZipCode,WDate,Temperature_Max,Temperature_Avg,Temperature_M,Dewpot_Max,...,Dewpot_M,Humidity_Max,Humidity_Avg,Humidity_M,Wdspeed_Max,Wdspeed_Avg,Wdspeed_M,Pressure_Max,Pressure_M,Precipitation_Total
0,KNYBRONX14,Bronx,Botanical Garden,NY,10458,1/1/2016,41.2,38.1,33.9,26.9,...,16.9,60,51.0,45,28.4,11.3,1.3,30.10,29.96,0.00
1,KNYBRONX14,Bronx,Botanical Garden,NY,10458,1/2/2016,39.4,35.2,32.4,19.3,...,14.0,56,48.0,42,25.3,10.1,0.0,30.11,29.95,0.00
2,KNYBRONX14,Bronx,Botanical Garden,NY,10458,1/3/2016,44.7,38.6,34.5,23.5,...,19.5,60,49.0,36,28.4,10.8,1.6,29.97,29.78,0.00
3,KNYBRONX14,Bronx,Botanical Garden,NY,10458,1/4/2016,35.6,25.9,12.1,23.8,...,-0.8,62,48.0,31,36.5,10.2,1.1,30.39,29.86,0.00
4,KNYBRONX14,Bronx,Botanical Garden,NY,10458,1/5/2016,28.8,18.5,10.0,6.5,...,-6.9,64,45.0,21,27.5,6.2,0.0,30.64,30.37,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10521,KNYNEWYO103,Staten Island,Richmondtown,NY,10308,10/3/2021,79.7,67.1,53.8,68.0,...,52.3,97,82.0,52,8.0,0.5,0.0,30.03,29.88,0.00
10522,KNYNEWYO103,Staten Island,Richmondtown,NY,10308,10/4/2021,75.2,68.8,62.6,69.3,...,60.8,96,91.0,80,7.0,0.2,0.0,30.05,29.86,0.14
10523,KNYNEWYO103,Staten Island,Richmondtown,NY,10308,10/5/2021,66.6,62.9,60.3,61.2,...,58.6,95,89.0,79,4.0,0.1,0.0,30.31,30.04,0.03
10524,KNYNEWYO103,Staten Island,Richmondtown,NY,10308,10/6/2021,70.3,65.2,61.9,62.2,...,58.3,94,85.0,74,5.0,0.1,0.0,30.37,30.29,0.00


In [4]:
df.isnull().values.any()

False

In [7]:
df["WeatherParamsID"] = df['WDate'].astype(str) +"-"+ df["Temperature_Avg"].astype(str) + "-" + df["ZipCode"].astype(str)
df['Year'] = pd.DatetimeIndex(df['WDate']).year
df['Month'] = pd.DatetimeIndex(df['WDate']).month
df['Day'] = pd.DatetimeIndex(df['WDate']).day
df["DateID"] = df['WDate'].astype(str) +"-"+ df["ZipCode"].astype(str)


In [10]:
df_address = df[["Borough","City","State","ZipCode","Station"]]

In [9]:
df_fact = df[['WeatherParamsID','ZipCode',"WDate","Temperature_Avg",'Dewpot_Avg','Humidity_Avg','Wdspeed_Avg','Precipitation_Total']]

In [11]:
df_address = df_address.drop_duplicates()
df_address

,Borough,City,State,ZipCode,Station
0,Bronx,Botanical Garden,NY,10458,KNYBRONX14
2107,Brooklyn,Dyker Heights,NY,11228,KNYBROOK54
4207,Manhattan,New York,NY,10018,KNYNEWYO343
6313,Queens,Jackson Heights,NY,11372,KNYJACKS2
8419,Staten Island,Richmondtown,NY,10306,KNYNEWYO103
10247,Staten Island,Richmondtown,NY,10308,KNYNEWYO103


In [24]:
df_params = df[['WeatherParamsID','Temperature_Avg','Temperature_Max','Temperature_M','Dewpot_Max','Dewpot_M','Humidity_Max','Humidity_M','Wdspeed_Max',
                'Wdspeed_M','Pressure_Max','Pressure_M']]

In [13]:
df_date = df[["DateID","WDate","Year","Month","Day"]]


In [14]:
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = "cis4400proj-370416-74e9d8391c22.json"

In [15]:
client = bigquery.Client()

In [16]:
dataset_name = 'weather'
table_id = dataset_name + '.address_dim_tbl'
table_id2 = dataset_name + '.facts_dim_tbl'
table_id3 = dataset_name + '.params_dim_tbl'
table_id4 = dataset_name + '.date_dim_tbl'

In [17]:
job_config = bigquery.LoadJobConfig(
    schema = [bigquery.SchemaField("Borough",bigquery.enums.SqlTypeNames.STRING)
             ], write_disposition = "WRITE_TRUNCATE")

In [27]:
job_config2 = bigquery.LoadJobConfig(
    schema = [bigquery.SchemaField("WeatherParamsID",bigquery.enums.SqlTypeNames.STRING)
             ], write_disposition = "WRITE_TRUNCATE")

In [19]:
job_config3 = bigquery.LoadJobConfig(
    schema = [bigquery.SchemaField("WeatherParamsID",bigquery.enums.SqlTypeNames.STRING)
             ], write_disposition = "WRITE_TRUNCATE")

In [20]:
job_config4 = bigquery.LoadJobConfig(
    schema = [bigquery.SchemaField("DateID",bigquery.enums.SqlTypeNames.STRING)
             ], write_disposition = "WRITE_TRUNCATE")

In [21]:
job = client.load_table_from_dataframe(
df_address, table_id, job_config=job_config)

job.result()

LoadJob<project=cis4400proj-370416, location=US, id=2fb3d3cf-8b5b-4f67-a73d-0b1ef12aee4f>

In [28]:
job2 = client.load_table_from_dataframe(
df_fact, table_id2, job_config=job_config2)

job.result()

LoadJob<project=cis4400proj-370416, location=US, id=2fb3d3cf-8b5b-4f67-a73d-0b1ef12aee4f>

In [23]:
job3 = client.load_table_from_dataframe(
df_params, table_id3, job_config=job_config3)

job.result()

LoadJob<project=cis4400proj-370416, location=US, id=2e220974-491b-45f2-bdad-d04f3225e954>

In [24]:
job3 = client.load_table_from_dataframe(
df_date, table_id4, job_config=job_config4)

job.result()

LoadJob<project=cis4400proj-370416, location=US, id=2e220974-491b-45f2-bdad-d04f3225e954>